
# OpenAI SDK + Agents SDK — Colab Starter

This notebook installs the official **OpenAI Python SDK** (`openai`) and the **OpenAI Agents SDK** (`openai-agents`), prompts you for your API key securely, and runs a quick test for both the **Responses API** and a **minimal Agent with a tool**.

**What you'll do:**
1. Install packages  
2. Enter your API key securely (not saved in the notebook)  
3. Test the Responses API  
4. Test the Agents SDK with a tiny tool


In [8]:

# 1) Install the SDKs
#!pip -q install --upgrade openai openai-agents
!pip install --upgrade --force-reinstall openai openai-agents


  Using cached openai-1.99.9-py3-none-any.whl.metadata (29 kB)
  Using cached openai_agents-0.2.7-py3-none-any.whl.metadata (12 kB)
  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached griffe-1.12.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached mcp-1.13.0-py3-none-any.whl.metadata (68 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached types_requests-2.32.4.20250809-py3-none-any.whl.metadata (2.0 kB)
  Using cached idna-3.10-py3-none-any.

In [1]:

# 2) Enter your API key securely
from getpass import getpass
import os

print("Enter your OpenAI API key (input hidden):")
os.environ["OPENAI_API_KEY"] = getpass()
assert os.environ["OPENAI_API_KEY"], "No API key entered."
print("✅ API key set (env var only for this runtime).")


Enter your OpenAI API key (input hidden):
··········
✅ API key set (env var only for this runtime).


In [2]:

# 3) Test the OpenAI SDK (Responses API)
from openai import OpenAI

client = OpenAI()
resp = client.responses.create(
    model="gpt-4.1-mini",
    input="Say 'SDK installed and ready' in one short sentence."
)
print("Model:", resp.model if hasattr(resp, "model") else "unknown")
print(resp.output_text)


Model: gpt-4.1-mini-2025-04-14
SDK installed and ready.


In [17]:
# 4) Test the OpenAI Agents SDK with a tiny tool
from agents import Agent, Runner # Explicitly import Agent and run

def add(a: int, b: int) -> int:
    """Add two integers and return the sum."""
    return a + b

assistant = Agent(
    name="MathAssistant",
    model="gpt-4.1-mini",
    instructions="You are a concise assistant. Use tools when helpful.",
    tools=[add],
)

result = Runner.run(assistant, "Use the add tool to add 2 and 5, then explain the result in one short sentence.")
# Newer SDKs return a structured result; print a friendly view:
try:
    print(result.output_text)
except Exception:
    print(result)

<coroutine object Runner.run at 0x7afd373ff370>


/tmp/ipython-input-4208906432.py:15: RuntimeWarning: coroutine 'Runner.run' was never awaited
  result = Runner.run(assistant, "Use the add tool to add 2 and 5, then explain the result in one short sentence.")


In [18]:
# # Make sure the package is installed & fresh
# !pip -q install --upgrade --force-reinstall openai-agents

from agents import Agent, Runner  # correct import for the Agents SDK

def add(a: int, b: int) -> int:
    return a + b

assistant = Agent(
    name="MathAssistant",
    model="gpt-4.1-mini",
    instructions="Be concise. Use tools when helpful.",
    tools=[add],
)

# ✅ No asyncio issues in notebooks
result = Runner.run_sync(assistant, "Use the add tool to add 2 and 5, then explain briefly.")
print(getattr(result, "final_output", getattr(result, "output_text", result)))


RuntimeError: This event loop is already running

In [19]:
from agents import Agent, Runner

def add(a: int, b: int) -> int:
    return a + b

assistant = Agent(
    name="MathAssistant",
    model="gpt-4.1-mini",
    instructions="Be concise. Use tools when helpful.",
    tools=[add],
)

async def main():
    result = await Runner.run(assistant, "Use the add tool to add 2 and 5, then explain briefly.")
    print(getattr(result, "final_output", getattr(result, "output_text", result)))

# In Colab/Jupyter, call with top-level await:
await main()
# (If top-level await isn't available, install and apply nest_asyncio, then use asyncio.run)


AttributeError: 'function' object has no attribute 'name'


### Notes
- You can switch `model` to another available one if needed.
- Your API key is only stored in the Colab runtime's environment for this session.
- To visualize agents or use voice/realtime extras, install optional extras, e.g.:
  ```bash
  pip install "openai-agents[viz]"
  pip install "openai-agents[voice]"
  ```
